# 1.Data cleaning

## 1.1 Setup

In [1]:
import re
import pandas as pd
from nltk.corpus import stopwords
import nltk
import spacy
import json

## 1.2 Loading Data

### Train data

In [2]:
# We load our train data into a dataFrame
trainDf = pd.read_excel('resources/referentiel_foodex.xlsx',sheet_name='Feuil1')
trainDf

,Désignation commerciale,Catégorie de référence
0,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao)
2,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme
...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits


### Test data

In [ ]:
# Convert the test.json into an Excel file to be easy to annoutate.
with open("resources/test.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
products = data["designations"]
testDf = pd.DataFrame({
    "Product": products,
    "Category": [""] * len(products)
})
testDf

,Product,Category
0,Lait au chocolat,
1,Poisson en chocolat,
2,Chocapic en poudre,
3,Cola light sans bulles,
4,Jus de pomme bio,
...,...,...
95,Crème brûlée vanille,
96,Macaron framboise,
97,Tarte tatin,
98,Clafoutis aux cerises,


## 1.3 Cleaning steps

### Duplicated rows

In [4]:
# Check if there are duplicated rows 
# No duplicated rows in train

duplicates = trainDf[trainDf.duplicated()]
duplicates

,Désignation commerciale,Catégorie de référence


In [5]:
# No duplicated rows in test

duplicates = testDf[testDf.duplicated()]
duplicates

,Product,Category


### Missing values

In [6]:
# No missing values in test
testDf.isnull().sum()

Product     0
Category    0
dtype: int64

In [8]:
# 3 missing values in train
trainDf.isnull().sum()

Désignation commerciale    0
Catégorie de référence     3
dtype: int64

In [9]:
# There 3 missing categories, for the moment we just remove them 
#-> (any row that has a missing value in a column will be dropped)

trainDf_Cleaned = trainDf.dropna()
trainDf_Cleaned = trainDf_Cleaned.rename(columns={
    'Désignation commerciale':'Product',
    'Catégorie de référence':'Category'})
trainDf_Cleaned

,Product,Category
0,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao)
2,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme
...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits


In [10]:
testDf_Cleaned = testDf

### Scientific names handeling

In [12]:
def remove_scientific_names(text):
    # Use regex to find and remove all text between parentheses (scientific names)
    cleaned_text = re.sub(r'\(.*?\)', '', text)
    return cleaned_text

In [13]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product"].apply(remove_scientific_names)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category"].apply(remove_scientific_names)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,Lait au chocolat,Chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),Poisson en chocolat,Produits de chocolat
2,Chocapic en poudre,Poudre de cacao,Chocapic en poudre,Poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo..."
4,Jus de pomme bio,Jus de pomme,Jus de pomme bio,Jus de pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",Crème brûlée vanille,"Collations, desserts et autres aliments"
96,Macaron framboise,Pâtisseries et gâteaux,Macaron framboise,Pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits,Tarte tatin,Tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits,Clafoutis aux cerises,Gâteau aux fruits


In [14]:
testDf_Cleaned["Product_clean"] = testDf_Cleaned["Product"].apply(remove_scientific_names)
testDf_Cleaned

,Product,Category,Product_clean
0,Lait au chocolat,,Lait au chocolat
1,Poisson en chocolat,,Poisson en chocolat
2,Chocapic en poudre,,Chocapic en poudre
3,Cola light sans bulles,,Cola light sans bulles
4,Jus de pomme bio,,Jus de pomme bio
...,...,...,...
95,Crème brûlée vanille,,Crème brûlée vanille
96,Macaron framboise,,Macaron framboise
97,Tarte tatin,,Tarte tatin
98,Clafoutis aux cerises,,Clafoutis aux cerises


### Special caracters and Lowercasing

In [15]:
def cleanText(text):
    text = re.sub(r"[^a-zA-Z0-9àâäéèêëîïôöùûüÿçœæÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇŒÆ\s]"," ",text) # Replace special caracters with white space.
    return text.lower() # lowercase

In [16]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product_clean"].apply(cleanText)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category_clean"].apply(cleanText)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait au chocolat,chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),poisson en chocolat,produits de chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic en poudre,poudre de cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola light sans bulles,boissons au cola caféiniques faibles en calo...
4,Jus de pomme bio,Jus de pomme,jus de pomme bio,jus de pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème brûlée vanille,collations desserts et autres aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries et gâteaux
97,Tarte tatin,Tarte aux fruits,tarte tatin,tarte aux fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis aux cerises,gâteau aux fruits


In [17]:
testDf_Cleaned["Product_clean"] = testDf_Cleaned["Product_clean"].apply(cleanText)
testDf_Cleaned

,Product,Category,Product_clean
0,Lait au chocolat,,lait au chocolat
1,Poisson en chocolat,,poisson en chocolat
2,Chocapic en poudre,,chocapic en poudre
3,Cola light sans bulles,,cola light sans bulles
4,Jus de pomme bio,,jus de pomme bio
...,...,...,...
95,Crème brûlée vanille,,crème brûlée vanille
96,Macaron framboise,,macaron framboise
97,Tarte tatin,,tarte tatin
98,Clafoutis aux cerises,,clafoutis aux cerises


### Stopwords removing 

In [18]:
# Download French stopwords if not already
nltk.download('stopwords')
french_stopwords = set(stopwords.words('french'))
len(french_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


157

In [19]:
def remove_french_stopwords(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return ' '.join([w for w in words if w not in french_stopwords])


In [20]:
# Apply to both columns
trainDf_Cleaned['Product_clean'] = trainDf_Cleaned['Product_clean'].apply(remove_french_stopwords)
trainDf_Cleaned['Category_clean'] = trainDf_Cleaned['Category_clean'].apply(remove_french_stopwords)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait chocolat,chocolat chaud
1,Poisson en chocolat,Produits de chocolat (cacao),poisson chocolat,produits chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic poudre,poudre cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola light sans bulles,boissons cola caféiniques faibles calories
4,Jus de pomme bio,Jus de pomme,jus pomme bio,jus pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème brûlée vanille,collations desserts autres aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries gâteaux
97,Tarte tatin,Tarte aux fruits,tarte tatin,tarte fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis cerises,gâteau fruits


In [21]:
testDf_Cleaned["Product_clean"] = testDf_Cleaned["Product_clean"].apply(remove_french_stopwords)
testDf_Cleaned

,Product,Category,Product_clean
0,Lait au chocolat,,lait chocolat
1,Poisson en chocolat,,poisson chocolat
2,Chocapic en poudre,,chocapic poudre
3,Cola light sans bulles,,cola light sans bulles
4,Jus de pomme bio,,jus pomme bio
...,...,...,...
95,Crème brûlée vanille,,crème brûlée vanille
96,Macaron framboise,,macaron framboise
97,Tarte tatin,,tarte tatin
98,Clafoutis aux cerises,,clafoutis cerises


### Keep only Nouns

use the following command to install the model :

```bash
uv run python -m spacy download fr_dep_news_trf 
```

In [22]:
nlp = spacy.load("fr_dep_news_trf")

In [23]:
def keep_nouns(text):
    doc = nlp(text)
    cleanedText = " ".join([token.text for token in doc if token.pos_ in ["NOUN","PROPN"] ])
    if len(cleanedText) > 0:
        return cleanedText
    return text

In [24]:
# Example 

print(keep_nouns("boissons au cola caféiniques faibles en "))

doc = nlp("boissons au cola caféiniques faibles en ")
for token in doc:
    print(token.text, token.pos_)

boissons cola
boissons NOUN
au ADP
cola NOUN
caféiniques ADJ
faibles ADJ
en ADP


In [25]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product_clean"].apply(keep_nouns)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category_clean"].apply(keep_nouns)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait chocolat,chocolat
1,Poisson en chocolat,Produits de chocolat (cacao),poisson chocolat,produits chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic poudre,poudre cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola bulles,boissons calories
4,Jus de pomme bio,Jus de pomme,jus pomme,jus pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème vanille,collations desserts aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries gâteaux
97,Tarte tatin,Tarte aux fruits,tarte,tarte fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis cerises,gâteau fruits


In [26]:
testDf_Cleaned["Product_clean"] = testDf_Cleaned["Product_clean"].apply(keep_nouns)
testDf_Cleaned

,Product,Category,Product_clean
0,Lait au chocolat,,lait chocolat
1,Poisson en chocolat,,poisson chocolat
2,Chocapic en poudre,,chocapic poudre
3,Cola light sans bulles,,cola bulles
4,Jus de pomme bio,,jus pomme
...,...,...,...
95,Crème brûlée vanille,,crème vanille
96,Macaron framboise,,macaron framboise
97,Tarte tatin,,tarte
98,Clafoutis aux cerises,,clafoutis cerises


### Redundant words handling

In [27]:
def remove_redundant_words(text):
    words = text.split()
    unique_words = set(words)
    cleaned_text = ' '.join(sorted(unique_words, key=words.index))
    return cleaned_text

In [28]:
trainDf_Cleaned["Product_clean"] = trainDf_Cleaned["Product_clean"].apply(remove_redundant_words)
trainDf_Cleaned["Category_clean"] = trainDf_Cleaned["Category_clean"].apply(remove_redundant_words)
trainDf_Cleaned

,Product,Category,Product_clean,Category_clean
0,Lait au chocolat,Chocolat chaud,lait chocolat,chocolat
1,Poisson en chocolat,Produits de chocolat (cacao),poisson chocolat,produits chocolat
2,Chocapic en poudre,Poudre de cacao,chocapic poudre,poudre cacao
3,Cola light sans bulles,"Boissons au cola, caféiniques, faibles en calo...",cola bulles,boissons calories
4,Jus de pomme bio,Jus de pomme,jus pomme,jus pomme
...,...,...,...,...
95,Crème brûlée vanille,"Collations, desserts et autres aliments",crème vanille,collations desserts aliments
96,Macaron framboise,Pâtisseries et gâteaux,macaron framboise,pâtisseries gâteaux
97,Tarte tatin,Tarte aux fruits,tarte,tarte fruits
98,Clafoutis aux cerises,Gâteau aux fruits,clafoutis cerises,gâteau fruits


In [29]:
testDf_Cleaned["Product_clean"] = testDf_Cleaned["Product_clean"].apply(remove_redundant_words)
testDf_Cleaned

,Product,Category,Product_clean
0,Lait au chocolat,,lait chocolat
1,Poisson en chocolat,,poisson chocolat
2,Chocapic en poudre,,chocapic poudre
3,Cola light sans bulles,,cola bulles
4,Jus de pomme bio,,jus pomme
...,...,...,...
95,Crème brûlée vanille,,crème vanille
96,Macaron framboise,,macaron framboise
97,Tarte tatin,,tarte
98,Clafoutis aux cerises,,clafoutis cerises


### Save the cleaned dataframe

In [30]:
trainDf_Cleaned.iloc[:, -2:].to_excel('data/train_cleaned.xlsx',index=False)
testDf_Cleaned.to_excel('data/test_cleaned.xlsx',index=False)

# 2.Classification

## 2.1 Evaluation function 

## 2.2 Keywords based Pre-Selection

### Setup

In [ ]:
import pandas as pd

In [ ]:
trainDf = pd.read_excel('data/train_cleaned.xlsx')
trainDf


### Get Candidates

In [ ]:
def get_Candidates(productName):
    categories = trainDf['Category_clean'].tolist()
    candidates = set()
    keywords = productName.strip().split()
    for keyword in keywords:
        for category in categories:
            if keyword in category.strip().split():
                candidates.add(category)
    return list(candidates)

def runAll():
    products = trainDf['Product_clean'].tolist()
    candidates_list = []
    for product in products:
        candidates = get_Candidates(product)
        candidates_list.append(candidates)
    trainDf['Candidate_categories'] = candidates_list
    return trainDf

In [ ]:
# Example
get_Candidates("purée pommes terre")

In [ ]:
train_with_candidates = runAll()

### Set predictions

If a product has only one candidate, there is no need for the refinement step, as the predicted category is already determined.

In [ ]:
train_with_candidates['Category_predicted'] = train_with_candidates['Candidate_categories'].apply(
    lambda x: x[0] if len(x) == 1 else ''
)
train_with_candidates.to_excel('data/train_with_candidates.xlsx',index=False)
train_with_candidates